In [11]:
!pip install "thinc>=8.0.0a0" ml_datasets "tqdm>=4.41"

In [12]:
from thinc.api import prefer_gpu
prefer_gpu()


False

In [27]:
import ml_datasets
from tqdm import tqdm
from thinc.api import fix_random_seed

In [28]:
fix_random_seed(0)

def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_y), (test_X, test_y) = data
    model.initialize(X=train_X[:5], Y=train_y[:5])
    for n in range(n_iter):
        loss = 0.0
        batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            d_loss = []
            for i in range(len(Yh)):
                d_loss.append(Yh[i] - Y[i])
                loss += ((Yh[i] - Y[i]) ** 2).sum()
            backprop(d_loss)
            model.finish_update(optimizer)
        score = evaluate(model, test_X, test_y, batch_size)
        print(f"{n}\t{loss:.2f}\t{score:.3f}")
        
def evaluate(model, test_X, test_Y, batch_size):
    correct = 0
    total = 0
    for X, Y in model.ops.multibatch(batch_size, test_X, test_Y):
        Yh = model.predict(X)
        for yh, y in zip(Yh, Y):
            correct += (y.argmax(axis=1) == yh.argmax(axis=1)).sum()
            total += y.shape[0]
    return float(correct / total)

## Composing the model in code

Here's the model definition, using the `>>` operator for the `chain` combinator.
The `strings2arrays` transform converts a sequence of strings to a list of arrays. `with_array` 
transforms sequences (the sequences of arrays) into a contiguous 2-dimensional array on the way into 
and out of the model it wraps. This means our model has the following signature: 
`Model[Sequence[str], Sequence[Array2d]]`.


In [1]:
from thinc.api import Model, chain, strings2arrays, with_array, HashEmbed, expand_window, Relu, Softmax, Adam, warmup_linear

width = 32
vector_width = 16
nr_classes = 17
learn_rate = 0.001
n_iter = 10
batch_size = 128

with Model.define_operators({">>": chain}):
    model = strings2arrays() >> with_array(
        HashEmbed(nO=width, nV=vector_width, column=0)
        >> expand_window(window_size=1)
        >> Relu(nO=width, nI=width * 3)
        >> Relu(nO=width, nI=width)
        >> Softmax(nO=nr_classes, nI=width)
    )
optimizer = Adam(learn_rate)

## Composing the model via config file

If we want to rebuild the model defined above in a config file, we first need to break down its structure:

* `chain` (any number of positional arguments)
  * `strings2arrays` (no arguments)
  * `with_array` (one argument **layer**)
    * **layer:** `chain` (any number of positional arguments)
      * `HashEmbed`
      * `expand_window`
      * `Relu`
      * `Relu`
      * `Softmax`

`chain` takes a variable number of positional arguments (the layers to compose). 
In the config, positional arguments can be expressed using `*` in the dot notation. For example, 
`model.layer` could describe a function passed to `model` as the argument `layer`, while `model.*.relu` 
defines a positional argument passed to `model`. The name of the argument, e.g. `relu` – doesn't matter 
in this case. It just needs to be unique.

> ⚠️ **Important note:** It is recommended to use a hybrid approach: wrap the model definition in a registered function
> and configure it via the config.

In [32]:
CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001
n_tags = 17

[training]
n_iter = 10
batch_size = 128

[model]
@layers = "chain.v1"

[model.*.strings2arrays]
@layers = "strings2arrays.v1"

[model.*.with_array]
@layers = "with_array.v1"

[model.*.with_array.layer]
@layers = "chain.v1"

[model.*.with_array.layer.*.hashembed]
@layers = "HashEmbed.v1"
nO = ${hyper_params:width}
nV = ${hyper_params:vector_width}
column = 0

[model.*.with_array.layer.*.expand_window]
@layers = "expand_window.v1"
window_size = 1

[model.*.with_array.layer.*.relu1]
@layers = "Relu.v1"
nO = ${hyper_params:width}
nI = 96

[model.*.with_array.layer.*.relu2]
@layers = "Relu.v1"
nO = ${hyper_params:width}
nI = ${hyper_params:width}

[model.*.with_array.layer.*.softmax]
@layers = "Softmax.v1"
nO = ${hyper_params:n_tags}
nI = ${hyper_params:width}

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""


from thinc.api import registry, Config

config = Config().from_str(CONFIG)
loaded_config = registry.make_from_config(config)

model = loaded_config["model"]
optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

{'nodes': [{'index': 0,
   'name': 'strings2arrays>>with_array-ints-getitem>>hashembed>>expand_window>>relu>>relu>>softmax',
   'dims': {'nO': None, 'nI': None},
   'refs': {}},
  {'index': 1, 'name': 'strings2arrays', 'dims': {}, 'refs': {}},
  {'index': 2,
   'name': 'with_array-ints-getitem>>hashembed>>expand_window>>relu>>relu>>softmax',
   'dims': {},
   'refs': {}},
  {'index': 3,
   'name': 'ints-getitem>>hashembed>>expand_window>>relu>>relu>>softmax',
   'dims': {'nO': None, 'nI': None},
   'refs': {}},
  {'index': 4,
   'name': 'ints-getitem>>hashembed',
   'dims': {'nO': None, 'nI': None},
   'refs': {}},
  {'index': 5, 'name': 'expand_window', 'dims': {}, 'refs': {}},
  {'index': 6, 'name': 'relu', 'dims': {'nO': 32, 'nI': 96}, 'refs': {}},
  {'index': 7, 'name': 'relu', 'dims': {'nO': 32, 'nI': 32}, 'refs': {}},
  {'index': 8, 'name': 'softmax', 'dims': {'nO': 17, 'nI': 32}, 'refs': {}},
  {'index': 9, 'name': 'ints-getitem', 'dims': {}, 'refs': {}},
  {'index': 10,
   'nam

In [33]:
data = ml_datasets.ud_ancora_pos_tags()
train_model(data, model, optimizer, n_iter, batch_size)

 12%|█▏        | 13/112 [00:00<00:00, 124.01it/s] 

0	394039.86	0.427


 12%|█▎        | 14/112 [00:00<00:00, 132.44it/s] 

1	293592.17	0.530


 12%|█▏        | 13/112 [00:00<00:00, 126.72it/s] 

...

 11%|█         | 12/112 [00:00<00:00, 117.31it/s] 

7	189556.96	0.676


 11%|█         | 12/112 [00:00<00:00, 117.56it/s] 

8	184881.38	0.686


9	180871.01	0.691


## Composing the model with code and config


In [18]:
import thinc
from thinc.api import Model, chain, strings2arrays, with_array, HashEmbed, expand_window, Relu, Softmax,\
    Adam, warmup_linear

@thinc.registry.layers("cnn_tagger.v1")
def create_cnn_tagger(width: int, vector_width: int, nr_classes: int = 17) -> Model:
    with Model.define_operators({">>": chain}):
        model = strings2arrays() >> with_array(
            HashEmbed(nO=width, nV=vector_width, column=0)
            >> expand_window(window_size=1)
            >> Relu(nO=width, nI=width * 3)
            >> Relu(nO=width, nI=width)
            >> Softmax(nO=nr_classes, nI=width)
        )
    return model

CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[model]
@layers = "cnn_tagger.v1"
width = ${hyper_params:width}
vector_width = ${hyper_params:vector_width}
nr_classes = 17

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""

loaded_config = registry.make_from_config(Config().from_str(CONFIG))

model = loaded_config["model"]
optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

data = ml_datasets.ud_ancora_pos_tags()

train_model(data, model, optimizer, n_iter, batch_size)

0	392620.84	0.403


1	298324.66	0.526


2	260850.97	0.579


3	240399.34	0.606


4	225486.50	0.629


5	214070.91	0.645


6	205949.52	0.656


7	199703.57	0.665


8	194561.25	0.674


9	189968.34	0.681
